In [22]:
import torch
from torch.utils.data import Dataset
import numpy as np
import segyio
import numpy as np
import os
import pandas as pd
import pywt
import matplotlib
import torch.nn.functional as F
from torch.utils.data import DataLoader
matplotlib.use('TkAgg')

def read_segy_as_data(segyfile, inline_row, xline_row, inline, xline, inline_start, xline_start):
    with segyio.open(segyfile,"r+",iline=inline_row, xline=xline_row) as sgydata:
        data = segyio.tools.cube(sgydata)
    head_data = data[inline-inline_start, xline-xline_start ,:]                  
    return head_data 

def cwts(cwts_scale, sampling_rate, data):
    #数据
    wavename = "mexh"  # 小波函数
    totalscal = cwts_scale + 1  # totalscal是对信号进行小波变换时所用尺度序列的长度(通常需要预先设定好)
    fc = pywt.central_frequency(wavename)  # 计算小波函数的中心频率
    cparam = 2 * fc * totalscal  # 常数c
    scales = cparam/np.arange(totalscal, 1, -1)  # 为使转换后的频率序列是一等差序列，尺度序列必须取为这一形式（也即小波尺度）
    cwtmatr = pywt.cwt(data, scales, wavename, 1.0/sampling_rate)[0]  # 连续小波变换模块
    feature_maps = cwtmatr
    return  feature_maps    

df1 = pd.read_excel('D:/桌面/SB/inf.xlsx',sheet_name = 'Sheet1', header = 0)
well_dict = df1.set_index('well_name').to_dict(orient='index')
for well_name, values in well_dict.items():
    print("Well Name:", well_name)
    print("Inline:", values['inline'])
    print("Xline:", values['xline'])
    print()  # 用于打印空行以区分不同的well_name

Well Name: 坨165
Inline: 996
Xline: 1210

Well Name: 坨168
Inline: 1084
Xline: 1207

Well Name: 坨198
Inline: 995
Xline: 1202

Well Name: 坨710
Inline: 1166
Xline: 1088

Well Name: 坨713
Inline: 1207
Xline: 1199

Well Name: 坨719
Inline: 1143
Xline: 1122

Well Name: 坨720
Inline: 1176
Xline: 1109

Well Name: 坨724
Inline: 1238
Xline: 1105

Well Name: 坨725
Inline: 1270
Xline: 1145

Well Name: 坨深1
Inline: 1235
Xline: 1212

Well Name: 坨深6
Inline: 1085
Xline: 1126

Well Name: 坨斜723
Inline: 1145
Xline: 1104

Well Name: 坨斜792
Inline: 1262
Xline: 1184

Well Name: 新利深1
Inline: 885
Xline: 1135



In [23]:
values = well_dict['坨165']
inline = values['inline']
xline = values['xline']
data_segy = read_segy_as_data('D:/桌面/SB/shengbei_pstm_cg.sgy', 9, 21, inline, xline, 640, 890)
data_segy.shape

(7000,)

In [24]:
features = cwts(128, 1000, data_segy)
features.shape

(128, 7000)

In [70]:
import os
import pandas as pd

folder_path = 'D:/桌面/SB/时深关系统计'
file_name_to_find = '坨165'
file_name_to_find = file_name_to_find + '.csv'

file_path = os.path.join(folder_path, file_name_to_find)
# 使用pandas读取csv文件
df = pd.read_csv(file_path)
# 打印数据框的内容
selected_columns = df.iloc[:, :2].values
depth = selected_columns[:,0]
time = selected_columns[:,1]

features = cwts(128, 1000, data_segy)
features = features[:, int(time[0]) : int(time[-1])]
features = torch.tensor(features)
features[:, 1 : 3].type

<function Tensor.type>

In [88]:
import os
import pandas as pd

folder_path = 'D:/桌面/SB/油水层统计'
file_name_to_find = '新利深1'
file_name_to_find = file_name_to_find + '.txt'
file_path = os.path.join(folder_path, file_name_to_find)
data = pd.read_csv(file_path, sep='\t', encoding = 'gbk')
selected_data = data[['顶深', '底深', '名称']]

# 将选定的数据转换为列表
data_list = selected_data.values.tolist()
# 汉字到数字的映射字典
mapping_dict = {
    '油层': 1,
    '油水同层': 2,
    '含油水层': 3,
    '水层': 4,
    '水淹层': 5,
    '弱水淹层': 6,
    '气水同层': 7,
    '干层': 8,
    '差油层': 9,
    '中水淹层': 10,
    '强水淹层': 11,
    '可疑气层': 12,
    '较强水淹层': 13,
    '泥岩裂缝储层': 14
}

# 替换列表中的汉字为数字
for entry in data_list:
    entry[2] = mapping_dict.get(entry[2], entry[2])

# 打印替换后的数据列表
data_list


[[2190.0, 2199.5, 4],
 [2208.0, 2219.3, 4],
 [2221.5, 2227.4, 4],
 [2470.2, 2471.8, 8],
 [2472.5, 2473.9, 8],
 [2481.4, 2483.4, 8],
 [3153.6, 3154.4, 8],
 [3155.6, 3157.0, 8],
 [3197.6, 3198.4, 1],
 [3230.5, 3231.4, 1],
 [3257.4, 3259.2, 8],
 [3263.4, 3266.7, 8],
 [3274.4, 3276.7, 8],
 [3279.4, 3280.6, 1],
 [3284.3, 3285.4, 1],
 [3286.7, 3289.7, 8],
 [3292.0, 3293.0, 8],
 [3299.6, 3302.5, 8],
 [3332.9, 3335.0, 8],
 [3338.4, 3339.2, 8],
 [3342.2, 3343.6, 8],
 [3345.7, 3346.6, 8],
 [3373.8, 3375.3, 8],
 [3376.7, 3377.4, 8],
 [3378.5, 3379.3, 8],
 [3379.3, 3380.0, 1],
 [3380.0, 3380.4, 8],
 [3384.2, 3386.2, 8],
 [3388.0, 3390.0, 8],
 [3397.1, 3399.2, 8],
 [3401.8, 3404.2, 8],
 [3412.4, 3414.6, 8],
 [3421.5, 3423.5, 8],
 [3436.5, 3437.7, 8],
 [3451.0, 3453.7, 8],
 [3461.7, 3463.0, 1],
 [3463.0, 3466.0, 8],
 [3468.8, 3469.7, 1],
 [3469.7, 3470.7, 8],
 [3475.5, 3476.7, 8],
 [3478.1, 3480.8, 8],
 [3485.0, 3488.7, 8],
 [3492.2, 3494.5, 8],
 [3496.0, 3497.3, 8],
 [3497.3, 3498.8, 1],
 [3498.8, 

In [94]:
import torch
from scipy.interpolate import interp1d

# 定义一维张量
a = torch.tensor([1, 2, 3.5, 4, 5])

# 构造插值函数
interp_func = interp1d(x=range(len(a)), y=a, kind='linear')

# 生成新的插值结果
new_indices = torch.linspace(0, len(a) - 1, 9)
interpolated_values = interp_func(new_indices)
interpolated_values


AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [96]:
import matplotlib.pyplot as plt
test_loss = np.load('D:/GD/test_loss.npy')
train_loss = np.load('D:/GD/train_loss.npy')
epochs = 50
epochs = range(1, epochs + 1)
# 绘制训练集和测试集损失函数图表
plt.plot(epochs, train_loss, 'b', label='Train Loss')
plt.plot(epochs, test_loss, 'r', label='Test Loss')
# 添加标题、轴标签和图例
plt.title('Training and Test Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [97]:
data_segy = np.array([1,2,3,4,5])
depth_segy = np.array([2,4,5,6,7])
plt.plot(depth_segy, data_segy)
plt.xlabel('Depth')
plt.ylabel('Data')
plt.title('SEGY Data Curve')
plt.grid(True)
plt.show()

In [102]:
# 随机生成特征图数据（这里使用 5x5 的大小）
feature_maps = np.random.rand(5, 5)

# 深度数组
depth_segy = np.array([2, 4, 5, 6, 7])

# 深度比例
depth_range = [depth_segy.min(), depth_segy.max()]

# 绘制图像
plt.figure(figsize=(8, 6))
plt.imshow(feature_maps, cmap='viridis', aspect='auto', interpolation='nearest', extent=[0, feature_maps.shape[1], depth_range[0], depth_range[1]])
plt.colorbar(label='Feature Value')
plt.xlabel('Feature Map Width')
plt.ylabel('Depth')
plt.title('Feature Maps at Different Depths')
plt.show()

In [101]:
# 给定的张量
data = np.array([8., 7., 7., 7., 7., 7., 7., 7., 7., 7., 8., 8., 8., 8., 8., 8., 8., 8.,
        8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
        8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 1., 1., 8., 8., 9., 9., 9.,
        9., 8., 8., 8., 8., 8., 1., 1., 1., 1., 8., 8., 8., 8., 8., 8., 8., 8.,
        8., 8., 8., 8., 8.])

# 使用不同的颜色代替每个数字
colors = plt.cm.viridis(np.linspace(0, 1, len(np.unique(data))))

# 绘制长方形图
for i, value in enumerate(data):
    plt.bar(i, 1, color=colors[int(value)])

# 隐藏坐标轴
plt.axis('off')

# 显示图形
plt.show()

IndexError: index 8 is out of bounds for axis 0 with size 4